In [1]:
import cv2, time, pandas
from datetime import datetime

first_frame = None
status_list = [None,None]
times = []

#DataFrame to Store the time Values during which object detection and movement appears
df=pandas.DataFrame(columns=["Start","End"])

video = cv2.VideoCapture(0)

while True:
    check, frame = video.read()
        
    # Status at the begning of the recording is zero as the object is not visible
    status = 0
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(21,21),0)

    # To store first image/frame of video
    if first_frame is None:
        first_frame=gray
        continue
        

     
    # Calculate the difference between the first frame and other frames
    delta_frame=cv2.absdiff(first_frame,gray)

    # Provides a threshold value, such that it will convert 
    # the difference value with less than 30 to black.
    # If the difference is greater than 30 it will convert 
    # those pixels to white
    thresh_frame=cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    # dilate the thresholded image to fill in holes, then find contours
	# on thresholded image
    thresh_frame=cv2.dilate(thresh_frame, None, iterations=2)
     
     # Define the contour area.
    # Basically, add the borders.
    (cnts,_)=cv2.findContours(thresh_frame.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Remove noises and shadows. Basically it will keep only that 
    # part white, which has area greater than 1000 pixels.
    for contour in cnts:
        if cv2.contourArea(contour) < 10000:
            continue
        # Status changes when object becomes visible
        status=1

        # Creates a rectangular box around the object in the frame
        (x, y, w, h)=cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)
    status_list.append(status)

    status_list=status_list[-2:]

    # Record datetime in a list when change occurs
    if status_list[-1]==1 and status_list[-2]==0:
        times.append(datetime.now())
    if status_list[-1]==0 and status_list[-2]==1:
        times.append(datetime.now())


    cv2.imshow("Gray Frame",gray)
    cv2.imshow("Delta Frame",delta_frame)
    cv2.imshow("Threshold Frame",thresh_frame)
    cv2.imshow("Color Frame",frame)

    key=cv2.waitKey(1)

    if key == ord('q'):
        if status == 1:
            times.append(datetime.now())
        break

print(status_list)
print(times)

for i in range(0, len(times), 2):
    # Store time values in DataFrame
    df = df.append({"Start": times[i],"End": times[i+1]}, ignore_index=True)

# Write a DataFrame to a CSV file
df.to_csv("Times.csv")

video.release()
cv2.destroyAllWindows

[1, 1]
[datetime.datetime(2019, 7, 6, 9, 11, 15, 277068)]


IndexError: list index out of range

In [ ]:
from motion_detector import df
from bokeh.plotting import figure, show, output_file, Figure
from bokeh.models import HoverTool, ColumnDataSource

df["Start_string"] = df["Start"].dt.strftime("%Y-%m-%d %H:%M:%S")
df["End_string"] = df["End"].dt.strftime("%Y-%m-%d %H:%M:%S")

cds = ColumnDataSource(df)

p = figure(x_axis_type='datetime', height=100, width=500, title="Motion Graph")
p.yaxis.minor_tick_line_color = None
p.ygrid[0].ticker.desired_num_ticks = 1

hover = HoverTool(tooltips=[("Start", "@Start_string"), ("End", "@End_string")])
p.add_tools(hover)

q = p.quad(left="Start", right="End", bottom=0, top=1, color="green", source=cds)

output_file("Graph1.html")
show(p)